# 2. Neural Style Transfer on your local machine

To make sure all the scripts that are working correctly, we need to test them on our local machine first. Specifically, we'll be testing the components that will be in our __flask app__ and our __scoring app__.

### Flask App
The __flask app__ is responsible for:
1. preprocessing the images into individual frames and extracting the audio file
2. adding references of the individual images to the Service Bus queue
3. waiting until style transfer has been applied to all images
4. postprocessing the style transfer images by stitching them together into a video and re-adding the audio clip

The __flask app__ contains the following files:
- `preprocess.py` 
- `add_images_to_queue.py` 
- `postprocess.py` 
- `util.py`
- `main.py`

`preprocess.py`, `add_images_to_queue.py`, and `postprocess.py` can be executed independently. This is to allow us to test each piece. The `main.py` file will call functions the aforementioned files. It will start by preprocessing the video then adding images to the service bus queue. Then it will wait for the messages in the queue to be dequeued and processed by another running process. (This occurs in the __scoring app__.) After all the images in the queue are processed, the `main.py` will start to postprocess the output frames, and recreate a new video, but this time with the style transfered frames.

### Scoring App
The __scoring app__ is responsible for:
1. polling the Service Bus queue for new images 
2. applying style transfer to those images and saving them back to storage

The __scoring app__ containers the following files:
- `main.py`
- `process_images_from_queue.py`
- `style_transfer.py`
- `util.py`

`main.py` starts the app by calling on the function in the `process_images_from_queue.py` file. This is responsible for continuously polling the service bus queue and checking for new messages. As images get pulled off the queue, the process will then call the stylize function in the `style_transfer.py` file which performs the actual style transfer. The generated style images are saved back to blob directly from this process. 

---

### Import package and load .env

In [1]:
from dotenv import set_key, get_key, find_dotenv, load_dotenv
from pathlib import Path
import json
import os

In [2]:
env_path = find_dotenv(raise_error_if_not_found=True)
load_dotenv(env_path)

True

### Preprocess video 
Run the preprocessing step on the video `orangutan.mp4` that is in blob. Specify the video (including the extention) and the mounted directory to access blob. 

In [3]:
!python flask_app/preprocess.py \
    --video orangutan.mp4 \
    --storage-mount-dir data

ffmpeg version 4.0 Copyright (c) 2000-2018 the FFmpeg developers
  built with gcc 7.2.0 (crosstool-NG fa8859cb)
  configuration: --prefix=/home/aperture/anaconda3/envs/bc_aks --cc=/opt/conda/conda-bld/ffmpeg_1531088893642/_build_env/bin/x86_64-conda_cos6-linux-gnu-cc --disable-doc --enable-shared --enable-static --enable-zlib --enable-pic --enable-gpl --enable-version3 --disable-nonfree --enable-hardcoded-tables --enable-avresample --enable-libfreetype --disable-openssl --disable-gnutls --enable-libvpx --enable-pthreads --enable-libopus --enable-postproc --disable-libx264
  libavutil      56. 14.100 / 56. 14.100
  libavcodec     58. 18.100 / 58. 18.100
  libavformat    58. 12.100 / 58. 12.100
  libavdevice    58.  3.100 / 58.  3.100
  libavfilter     7. 16.100 /  7. 16.100
  libavresample   4.  0.  0 /  4.  0.  0
  libswscale      5.  1.100 /  5.  1.100
  libswresample   3.  1.100 /  3.  1.100
  libpostproc    55.  1.100 / 55.  1.100
Input #0, mov,mp4,m4a,3gp,3g2,mj2, from 'data/orangu

### Add messages to queue
The preprocessing step splits the video into individual frames. Now add those frames to the queue to be processed later.

In [4]:
!python flask_app/add_images_to_queue.py \
    --video-name orangutan \
    --storage-mount-dir data

### Run the main style transfer process
The `scoring_app/main.py` file will dequeue images from the queue and apply style transfer to the images.

In [5]:
!python scoring_app/main.py --terminate

2019-07-17 03:56:52,326 [root:process_images_from_queue.py:34] DEBUG - Start listening to queue 'cambridgequ' on service bus...
2019-07-17 03:56:52,326 [root:process_images_from_queue.py:39] DEBUG - Peek queue...
2019-07-17 03:56:52,396 [root:process_images_from_queue.py:75] DEBUG - Queue message body: {'input_frame': '000001_frame.jpg', 'video_name': 'orangutan'}
2019-07-17 03:56:52,397 [root:process_images_from_queue.py:78] DEBUG - Starting style transfer on data/orangutan/input_frames/000001_frame.jpg
2019-07-17 03:56:52,560 [root:style_transfer.py:145] DEBUG - Processing data/orangutan/input_frames/000001_frame.jpg
2019-07-17 03:56:58,478 [root:process_images_from_queue.py:87] DEBUG - Finished style transfer on data/orangutan/input_frames/000001_frame.jpg
2019-07-17 03:56:58,479 [root:process_images_from_queue.py:90] DEBUG - Deleting queue message...
2019-07-17 03:56:58,545 [root:process_images_from_queue.py:39] DEBUG - Peek queue...
2019-07-17 03:56:58,557 [root:process_images_fro

2019-07-17 03:58:00,498 [root:process_images_from_queue.py:39] DEBUG - Peek queue...
2019-07-17 03:58:00,510 [root:process_images_from_queue.py:75] DEBUG - Queue message body: {'input_frame': '000012_frame.jpg', 'video_name': 'orangutan'}
2019-07-17 03:58:00,510 [root:process_images_from_queue.py:78] DEBUG - Starting style transfer on data/orangutan/input_frames/000012_frame.jpg
2019-07-17 03:58:00,543 [root:style_transfer.py:145] DEBUG - Processing data/orangutan/input_frames/000012_frame.jpg
2019-07-17 03:58:06,003 [root:process_images_from_queue.py:87] DEBUG - Finished style transfer on data/orangutan/input_frames/000012_frame.jpg
2019-07-17 03:58:06,003 [root:process_images_from_queue.py:90] DEBUG - Deleting queue message...
2019-07-17 03:58:06,050 [root:process_images_from_queue.py:39] DEBUG - Peek queue...
2019-07-17 03:58:06,062 [root:process_images_from_queue.py:75] DEBUG - Queue message body: {'input_frame': '000013_frame.jpg', 'video_name': 'orangutan'}
2019-07-17 03:58:06,06

2019-07-17 03:59:03,530 [root:style_transfer.py:145] DEBUG - Processing data/orangutan/input_frames/000023_frame.jpg
2019-07-17 03:59:09,236 [root:process_images_from_queue.py:87] DEBUG - Finished style transfer on data/orangutan/input_frames/000023_frame.jpg
2019-07-17 03:59:09,237 [root:process_images_from_queue.py:90] DEBUG - Deleting queue message...
2019-07-17 03:59:09,327 [root:process_images_from_queue.py:39] DEBUG - Peek queue...
2019-07-17 03:59:09,340 [root:process_images_from_queue.py:75] DEBUG - Queue message body: {'input_frame': '000024_frame.jpg', 'video_name': 'orangutan'}
2019-07-17 03:59:09,340 [root:process_images_from_queue.py:78] DEBUG - Starting style transfer on data/orangutan/input_frames/000024_frame.jpg
2019-07-17 03:59:09,394 [root:style_transfer.py:145] DEBUG - Processing data/orangutan/input_frames/000024_frame.jpg
2019-07-17 03:59:15,634 [root:process_images_from_queue.py:87] DEBUG - Finished style transfer on data/orangutan/input_frames/000024_frame.jpg
2

2019-07-17 04:00:11,346 [root:process_images_from_queue.py:87] DEBUG - Finished style transfer on data/orangutan/input_frames/000034_frame.jpg
2019-07-17 04:00:11,347 [root:process_images_from_queue.py:90] DEBUG - Deleting queue message...
2019-07-17 04:00:11,455 [root:process_images_from_queue.py:39] DEBUG - Peek queue...
2019-07-17 04:00:11,467 [root:process_images_from_queue.py:75] DEBUG - Queue message body: {'input_frame': '000035_frame.jpg', 'video_name': 'orangutan'}
2019-07-17 04:00:11,467 [root:process_images_from_queue.py:78] DEBUG - Starting style transfer on data/orangutan/input_frames/000035_frame.jpg
2019-07-17 04:00:11,496 [root:style_transfer.py:145] DEBUG - Processing data/orangutan/input_frames/000035_frame.jpg
2019-07-17 04:00:16,974 [root:process_images_from_queue.py:87] DEBUG - Finished style transfer on data/orangutan/input_frames/000035_frame.jpg
2019-07-17 04:00:16,974 [root:process_images_from_queue.py:90] DEBUG - Deleting queue message...
2019-07-17 04:00:17,0

2019-07-17 04:01:12,902 [root:process_images_from_queue.py:39] DEBUG - Peek queue...
2019-07-17 04:01:12,914 [root:process_images_from_queue.py:75] DEBUG - Queue message body: {'input_frame': '000046_frame.jpg', 'video_name': 'orangutan'}
2019-07-17 04:01:12,914 [root:process_images_from_queue.py:78] DEBUG - Starting style transfer on data/orangutan/input_frames/000046_frame.jpg
2019-07-17 04:01:12,942 [root:style_transfer.py:145] DEBUG - Processing data/orangutan/input_frames/000046_frame.jpg
2019-07-17 04:01:18,403 [root:process_images_from_queue.py:87] DEBUG - Finished style transfer on data/orangutan/input_frames/000046_frame.jpg
2019-07-17 04:01:18,403 [root:process_images_from_queue.py:90] DEBUG - Deleting queue message...
2019-07-17 04:01:18,479 [root:process_images_from_queue.py:39] DEBUG - Peek queue...
2019-07-17 04:01:18,491 [root:process_images_from_queue.py:75] DEBUG - Queue message body: {'input_frame': '000047_frame.jpg', 'video_name': 'orangutan'}
2019-07-17 04:01:18,49

...

2019-07-17 05:09:11,601 [root:style_transfer.py:145] DEBUG - Processing data/orangutan/input_frames/000772_frame.jpg
2019-07-17 05:09:17,075 [root:process_images_from_queue.py:87] DEBUG - Finished style transfer on data/orangutan/input_frames/000772_frame.jpg
2019-07-17 05:09:17,075 [root:process_images_from_queue.py:90] DEBUG - Deleting queue message...
2019-07-17 05:09:17,104 [root:process_images_from_queue.py:39] DEBUG - Peek queue...
2019-07-17 05:09:17,115 [root:process_images_from_queue.py:75] DEBUG - Queue message body: {'input_frame': '000773_frame.jpg', 'video_name': 'orangutan'}
2019-07-17 05:09:17,115 [root:process_images_from_queue.py:78] DEBUG - Starting style transfer on data/orangutan/input_frames/000773_frame.jpg
2019-07-17 05:09:17,144 [root:style_transfer.py:145] DEBUG - Processing data/orangutan/input_frames/000773_frame.jpg
2019-07-17 05:09:22,563 [root:process_images_from_queue.py:87] DEBUG - Finished style transfer on data/orangutan/input_frames/000773_frame.jpg
2

2019-07-17 05:10:18,281 [root:process_images_from_queue.py:87] DEBUG - Finished style transfer on data/orangutan/input_frames/000783_frame.jpg
2019-07-17 05:10:18,281 [root:process_images_from_queue.py:90] DEBUG - Deleting queue message...
2019-07-17 05:10:18,324 [root:process_images_from_queue.py:39] DEBUG - Peek queue...
2019-07-17 05:10:18,335 [root:process_images_from_queue.py:75] DEBUG - Queue message body: {'input_frame': '000784_frame.jpg', 'video_name': 'orangutan'}
2019-07-17 05:10:18,335 [root:process_images_from_queue.py:78] DEBUG - Starting style transfer on data/orangutan/input_frames/000784_frame.jpg
2019-07-17 05:10:18,366 [root:style_transfer.py:145] DEBUG - Processing data/orangutan/input_frames/000784_frame.jpg
2019-07-17 05:10:23,813 [root:process_images_from_queue.py:87] DEBUG - Finished style transfer on data/orangutan/input_frames/000784_frame.jpg
2019-07-17 05:10:23,813 [root:process_images_from_queue.py:90] DEBUG - Deleting queue message...
2019-07-17 05:10:23,8

2019-07-17 05:11:19,441 [root:process_images_from_queue.py:75] DEBUG - Queue message body: {'input_frame': '000795_frame.jpg', 'video_name': 'orangutan'}
2019-07-17 05:11:19,441 [root:process_images_from_queue.py:78] DEBUG - Starting style transfer on data/orangutan/input_frames/000795_frame.jpg
2019-07-17 05:11:19,469 [root:style_transfer.py:145] DEBUG - Processing data/orangutan/input_frames/000795_frame.jpg
2019-07-17 05:11:24,926 [root:process_images_from_queue.py:87] DEBUG - Finished style transfer on data/orangutan/input_frames/000795_frame.jpg
2019-07-17 05:11:24,926 [root:process_images_from_queue.py:90] DEBUG - Deleting queue message...
2019-07-17 05:11:25,016 [root:process_images_from_queue.py:39] DEBUG - Peek queue...
2019-07-17 05:11:25,027 [root:process_images_from_queue.py:75] DEBUG - Queue message body: {'input_frame': '000796_frame.jpg', 'video_name': 'orangutan'}
2019-07-17 05:11:25,028 [root:process_images_from_queue.py:78] DEBUG - Starting style transfer on data/oran

2019-07-17 05:12:20,756 [root:style_transfer.py:145] DEBUG - Processing data/orangutan/input_frames/000806_frame.jpg
2019-07-17 05:12:26,171 [root:process_images_from_queue.py:87] DEBUG - Finished style transfer on data/orangutan/input_frames/000806_frame.jpg
2019-07-17 05:12:26,172 [root:process_images_from_queue.py:90] DEBUG - Deleting queue message...
2019-07-17 05:12:26,216 [root:process_images_from_queue.py:39] DEBUG - Peek queue...
2019-07-17 05:12:26,227 [root:process_images_from_queue.py:75] DEBUG - Queue message body: {'input_frame': '000807_frame.jpg', 'video_name': 'orangutan'}
2019-07-17 05:12:26,228 [root:process_images_from_queue.py:78] DEBUG - Starting style transfer on data/orangutan/input_frames/000807_frame.jpg
2019-07-17 05:12:26,257 [root:style_transfer.py:145] DEBUG - Processing data/orangutan/input_frames/000807_frame.jpg
2019-07-17 05:12:31,676 [root:process_images_from_queue.py:87] DEBUG - Finished style transfer on data/orangutan/input_frames/000807_frame.jpg
2

2019-07-17 05:13:27,256 [root:process_images_from_queue.py:87] DEBUG - Finished style transfer on data/orangutan/input_frames/000817_frame.jpg
2019-07-17 05:13:27,257 [root:process_images_from_queue.py:90] DEBUG - Deleting queue message...
2019-07-17 05:13:27,358 [root:process_images_from_queue.py:39] DEBUG - Peek queue...
2019-07-17 05:13:27,370 [root:process_images_from_queue.py:75] DEBUG - Queue message body: {'input_frame': '000818_frame.jpg', 'video_name': 'orangutan'}
2019-07-17 05:13:27,370 [root:process_images_from_queue.py:78] DEBUG - Starting style transfer on data/orangutan/input_frames/000818_frame.jpg
2019-07-17 05:13:27,401 [root:style_transfer.py:145] DEBUG - Processing data/orangutan/input_frames/000818_frame.jpg
2019-07-17 05:13:32,845 [root:process_images_from_queue.py:87] DEBUG - Finished style transfer on data/orangutan/input_frames/000818_frame.jpg
2019-07-17 05:13:32,845 [root:process_images_from_queue.py:90] DEBUG - Deleting queue message...
2019-07-17 05:13:32,8

### Postprocess the frames
Now that the frames have been processed, we can stitch then back together and reattach the audio file using the `flask_app/postprocess.py` file.

In [6]:
!python flask_app/postprocess.py \
    --video-name orangutan \
    --storage-mount-dir data

ffmpeg version 4.0 Copyright (c) 2000-2018 the FFmpeg developers
  built with gcc 7.2.0 (crosstool-NG fa8859cb)
  configuration: --prefix=/home/aperture/anaconda3/envs/bc_aks --cc=/opt/conda/conda-bld/ffmpeg_1531088893642/_build_env/bin/x86_64-conda_cos6-linux-gnu-cc --disable-doc --enable-shared --enable-static --enable-zlib --enable-pic --enable-gpl --enable-version3 --disable-nonfree --enable-hardcoded-tables --enable-avresample --enable-libfreetype --disable-openssl --disable-gnutls --enable-libvpx --enable-pthreads --enable-libopus --enable-postproc --disable-libx264
  libavutil      56. 14.100 / 56. 14.100
  libavcodec     58. 18.100 / 58. 18.100
  libavformat    58. 12.100 / 58. 12.100
  libavdevice    58.  3.100 / 58.  3.100
  libavfilter     7. 16.100 /  7. 16.100
  libavresample   4.  0.  0 /  4.  0.  0
  libswscale      5.  1.100 /  5.  1.100
  libswresample   3.  1.100 /  3.  1.100
  libpostproc    55.  1.100 / 55.  1.100
Input #0, image2, from 'data/orangutan/output_frames

Play the video! It will be called `{video_name}_processed.mp4`

In [7]:
%%HTML
<video width="320" height="240" controls>
  <source src="data/orangutan/orangutan_processed.mp4" type="video/mp4">
</video>

### Additional commands to monitor
Use these additional commands to monitor.

```bash
# checkout how many elements are in the queue
!az servicebus queue show \
    --name {get_key(env_path, "SB_QUEUE")} \
    --namespace-name {get_key(env_path, "SB_NAMESPACE")} \
    --resource-group {get_key(env_path, "RESOURCE_GROUP")} \
    --query "countDetails.activeMessageCount"
```

---

Continue to the next [notebook](/notebooks/03_create_aks_cluster.ipynb).